## Memoria - ConversationBufferMemory


In [16]:
# Primero importamos las bibliotecas necesarias:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryBufferMemory

from dotenv import load_dotenv
import os

In [2]:
# Cargar variables de entorno
load_dotenv()
# Configurar el motor de OpenAI
engine = "gpt-3.5-turbo"
api_key=os.getenv("OPENAI_API_KEY")

In [3]:
llm = ChatOpenAI(temperature=0.0, model=engine, openai_api_key=api_key)
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False # Si ponemos verbose=True devuelve datos de la plantilla
)

In [4]:
conversation.predict(input="Hola, mi nombre es Carlos")

'¡Hola Carlos! ¿Cómo puedo ayudarte hoy?'

In [5]:
conversation.predict(input="¿Cuál es la capital de Honduras?")

'La capital de Honduras es Tegucigalpa.'

In [6]:
conversation.predict(input="¿Cuál es mi nombre?")

'Tu nombre es Carlos.'

In [7]:
print(memory.buffer)

Human: Hola, mi nombre es Carlos
AI: ¡Hola Carlos! ¿Cómo puedo ayudarte hoy?
Human: ¿Cuál es la capital de Honduras?
AI: La capital de Honduras es Tegucigalpa.
Human: ¿Cuál es mi nombre?
AI: Tu nombre es Carlos.


## ConversationBufferWindowMemory

In [10]:
llm = ChatOpenAI(temperature=0.0, model=engine, openai_api_key=api_key)
## USamos conversationBufferWindowMemory para almacenar el contexto de la conversación

memory = ConversationBufferWindowMemory(k=1) # Numero de interacciones en AI y usuario

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False # Si usas True podrás ver el prompt real que se está pasando al modelo

)

In [11]:
conversation.predict(input="Hola, mi nombre es Carlos")

'¡Hola Carlos! ¿Cómo puedo ayudarte hoy?'

In [12]:
conversation.predict(input="¿Cuál es la capital de Honduras?")

'La capital de Honduras es Tegucigalpa.'

In [13]:
conversation.predict(input="¿Cuál es mi nombre") # No lo recordará ya que k=1. Cambia k=2 para ver que funciona

'Lo siento, no tengo acceso a tu información personal, por lo que no sé cuál es tu nombre.'

In [14]:
## ConversationSummaryMemory

In [17]:
# creamos un texto
schedule = "Hay una reunión a las 8 a. m. con tu equipo de producto. \
Necesitarás tener preparada tu presentación de PowerPoint. \
De 9 a. m. a 12 p. m. tienes tiempo para trabajar en tu proyecto LangChain, \
que avanzará rápidamente porque LangChain es una herramienta muy potente.\
Al mediodía, almuerzo en el restaurante italiano con un cliente que viene conduciendo\
desde más de una hora de distancia para reunirse contigo y entender lo último en IA.\
Asegúrate de traer tu portátil para mostrar la última demo de LLM."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hola"}, {"output": "¿Qué hay?"})
memory.save_context({"input": "No mucho, por aquí..."},
                    {"output": "Guay"})
memory.save_context({"input": "Qué hay en la agenda hoy?"},
                    {"output": f"{schedule}"})

In [18]:
memory.load_memory_variables({})

{'history': 'System: The human and AI engage in a conversation about their daily agenda. The AI informs the human about a morning meeting with their product team and the need to prepare a PowerPoint presentation. The AI also mentions a project called LangChain that will progress quickly due to its powerful capabilities. The AI then mentions a lunch meeting with a client who has traveled a long distance to discuss the latest in artificial intelligence. The AI advises the human to bring their laptop to showcase the latest demo of LLM.'}

In [19]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [20]:
conversation.predict(input="Qué hay en la agenda para hoy?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in a conversation about their daily agenda. The AI informs the human about a morning meeting with their product team and the need to prepare a PowerPoint presentation. The AI also mentions a project called LangChain that will progress quickly due to its powerful capabilities. The AI then mentions a lunch meeting with a client who has traveled a long distance to discuss the latest in artificial intelligence. The AI advises the human to bring their laptop to showcase the latest demo of LLM.
Human: Qué hay en la agenda para hoy?
AI:

> Finished chain.


'Hola! En la agenda de hoy tenemos una reunión por la mañana con el equipo de productos, donde necesitaremos preparar una presentación en PowerPoint. También tenemos un proyecto llamado LangChain que está avanzando rápidamente debido a sus poderosas capacidades. Después, tenemos una reunión de almuerzo con un cliente que ha viajado desde lejos para discutir las últimas novedades en inteligencia artificial. Te aconsejo que traigas tu laptop para mostrar la última demostración de LLM. ¿Hay algo más en lo que pueda ayudarte?'